In [1]:
import os
import sys
import csv
import datetime
import shutil
import random
import glob
import pandas as pd
import numpy as np

#### add uniprot ID and gene symbol based on Ensembl ID.

In [2]:
uniprot_file = "/data/database/UniProt/idmapping/HUMAN_9606_idmapping_selected.tab"
ncbi_file = "/data/database/UniProt/idmapping/Homo_sapiens.gene_info"
gff = "/data/database/GRCh38/GENCODE/gencode.v27.annotation.gff3"
# pqtl = "/home/galaxy/project/m6AQTL/data/pqtl/total_data/Final_snp_from_paper.txt"
pqtl_file = "/home/lzg/1906/QTL_data/unfiltered/pQTL_paper.txt"
result_file = "/home/galaxy/project/m6AQTL/data/pqtl/total_data/pQTL.txt"

In [3]:
df_u = pd.read_table(uniprot_file, sep="\t")
ensembl_uniprot_dict = dict(zip(df_u.iloc[:, -4], df_u.iloc[:, 0]))
ens_uni_dict = {}
for ensembl, uniprot in ensembl_uniprot_dict.items():
    if str(ensembl).startswith("ENSG"):
        ensembl_list = ensembl.split("; ")
        for x in ensembl_list:
            ens_uni_dict[x] = uniprot
print("\t" + ens_uni_dict["ENSG00000108953"])  # test

	K7EM20


/home/galaxy/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
# df_n = pd.read_table(ncbi_file, sep="\t")
# ensembl_symbol_dict = dict(zip(df_n["dbXrefs"], df_n["Symbol"]))
# ens_sym_dict = {}
# for ensembl, symbol in ensembl_symbol_dict.items():
#     if "Ensembl" in ensembl:
#         ensembl = ensembl.split("Ensembl:")[1].strip()
#         ens_sym_dict[ensembl] = symbol
# print("\t" + ens_sym_dict["ENSG00000108953"])

	YWHAE


In [4]:
# To replace ncbi_file by using gff
df_g = pd.read_table(gff, sep="\t", comment="#", header=None)
df_g.head()

,0,1,2,3,4,5,6,7,8
0,chr1,HAVANA,gene,11869,14409,.,+,.,ID=ENSG00000223972.5;gene_id=ENSG00000223972.5...
1,chr1,HAVANA,transcript,11869,14409,.,+,.,ID=ENST00000456328.2;Parent=ENSG00000223972.5;...
2,chr1,HAVANA,exon,11869,12227,.,+,.,ID=exon:ENST00000456328.2:1;Parent=ENST0000045...
3,chr1,HAVANA,exon,12613,12721,.,+,.,ID=exon:ENST00000456328.2:2;Parent=ENST0000045...
4,chr1,HAVANA,exon,13221,14409,.,+,.,ID=exon:ENST00000456328.2:3;Parent=ENST0000045...


In [5]:
df_gene = df_g[df_g.iloc[:, 2] == "gene"]
# print(df_gene.head())
df_gene["Ensembl_ID"] = df_gene.iloc[:, 8].str.split("ID=").str[1].str.split(".").str[0]
df_gene["Symbol"] = df_gene.iloc[:, 8].str.split("gene_name=").str[1].str.split(";").str[0]
ens_sym_dict = dict(zip(df_gene["Ensembl_ID"], df_gene["Symbol"]))
print("\t" + ens_sym_dict["ENSG00000108953"]) # test

/home/galaxy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


	YWHAE


/home/galaxy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [6]:
df_p = pd.read_table(pqtl_file, sep="\t", skiprows=1)
print(df_p.head())
ensembl_list = df_p["Ensembl_ID"].tolist()
old_uniprot_list, old_symbol_list = df_p["Uniprot_ID"].tolist(), df_p["Gene_Name"].tolist()
uniprot_list, symbol_list = [], []
for i in range(len(ensembl_list)):
    ensembl = ensembl_list[i]
    if ensembl in ens_uni_dict:
        uniprot_list.append(ens_uni_dict[ensembl])
    elif old_uniprot_list[i] != "":
        uniprot_list.append(old_uniprot_list[i])
    else:
        uniprot_list.append(np.nan)
    if ensembl in ens_sym_dict:
        symbol_list.append(ens_sym_dict[ensembl])
    elif old_symbol_list[i] != "":
        symbol_list.append(old_symbol_list[i])
    else:
        symbol_list.append(np.nan)
# print(len(ensembl_list), len(symbol_list), len(uniprot_list))
df_p["Gene_Name"] = symbol_list
df_p["Protein_Name"] = symbol_list
df_p["Uniprot_ID"] = uniprot_list
print(df_p.head())
df_p.to_csv(result_file, sep="\t", index=False)

        Ensembl_ID Gene_Name Uniprot_ID Protein_Name         RS_ID  REF  ALT  \
0  ENSG00000120948    TARDBP        NaN       TARDBP  rs1471344782  NaN  NaN   
1  ENSG00000168610     STAT3        NaN       STAT3A   rs993990873  NaN  NaN   
2  ENSG00000100906    NFKBIA        NaN     IkBalpha  rs1477467102  NaN  NaN   
3  ENSG00000188342    GTF2F2        NaN       GTF2F2  rs1051246129  NaN  NaN   
4  ENSG00000270647     TAF15        NaN        TAF15  rs1452672552  NaN  NaN   

  pQTL_type Cutoff        Pvalue Corrected_pvalue Measure     Value  \
0     trans   1 Mb  3.156591e-08            < 0.2      R2  0.224095   
1       NaN    NaN  4.552463e-05              NaN       R  0.333490   
2       NaN    NaN  1.834424e-04              NaN       R -0.419501   
3       NaN    NaN  5.439675e-05              NaN       R  0.360772   
4       NaN    NaN  2.251364e-05              NaN       R -0.473694   

                     Cell_or_tissue     Condition  Experimental_tag PubMed_ID  
0  lymphobla